<a href="https://colab.research.google.com/github/lahirukavinda/ADL-AI-Summit---Pre-Hackathon/blob/main/ADLHack_SyntheticDataGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthetic Data Generator

Tested for XGBoost and MLP (both fails)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import random

df = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset.csv', error_bad_lines=False).fillna(0)
df.shape[0]

10500

In [4]:
df.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0,PKG2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,PKG6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0,PKG4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0,PKG4


In [5]:
x = df.drop('next_month_plan', axis='columns')
x = x.drop('primary_identifier', axis='columns')

y = pd.DataFrame()
y['next_month_plan'] = df['next_month_plan'].str.replace("PKG",'').astype(int)
y.head()

,next_month_plan
0,2
1,1
2,6
3,4
4,4


In [6]:
from sklearn.preprocessing import LabelEncoder

x["device_type"] = LabelEncoder().fit_transform(x["device_type"].astype(str))
x["device_category"] = LabelEncoder().fit_transform(x["device_category"].astype(str))
x["gender"] = LabelEncoder().fit_transform(x["gender"].astype(str))
x["district_name"] = LabelEncoder().fit_transform(x["district_name"].astype(str))
x["age_group"] = LabelEncoder().fit_transform(x["age_group"].astype(str))
x.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,3,5,2,11,3,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0
1,3,5,2,5,3,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0
2,3,5,2,10,2,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0
3,3,5,1,4,3,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0
4,3,5,1,10,4,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0


In [7]:
# Synthetic Data Generator

NUM_DATA_POINTS = 100000
FEATURE_LABELS = ['device_type','device_category','gender','district_name','age_group','network_stay','average_monthly_bill_amount','dusage_sum','dusage_min','dusage_max','dusage_avg','dusage_days','dusage_stddev','vusage_onnet_sum','vusage_onnet_max','vusage_onnet_min','vusage_onnet_days','vusage_onnet_avg','vusage_onnet_stddev','vusage_offnet_sum','vusage_offnet_max','vusage_offnet_min','vusage_offnet_days','vusage_offnet_avg','vusage_offnet_stddev','number_of_fixed_bb_accounts','number_of_iptv_accounts','add_on_tot_rental','add_on_count','next_month_plan']

NUM_FEATURES = len(x.columns) # 30
NUM_COMPONENTS = 8 # y['next_month_plan'].max()

def get_feature_val_range(x, y) :
    min_list = []
    max_list = []
    for i in range(1, NUM_COMPONENTS+1):
        x_filtered = x[y['next_month_plan']==i]
        #print(len(x_filtered.index))

        min_list_comp = []
        max_list_comp = []
        for column in x_filtered:
            feature_avg = x_filtered[column].mean()
            feature_min = x_filtered[column].min()
            feature_max = x_filtered[column].max()
            min_val = feature_min - abs(feature_max - feature_avg)
            max_val = feature_max + abs(feature_avg - feature_min)

            #min_range, max_range = max(0, min_val), max(0, max_val)
            min_list_comp.append(min_val)
            max_list_comp.append(max_val)
        
        min_list.append(min_list_comp)
        max_list.append(max_list_comp)

    return min_list, max_list

def generate_one_data_point(min_val_list, max_val_list) : 
    rand_comp = random.randint(1, NUM_COMPONENTS)

    data_point = []
    for i in range(NUM_FEATURES) :
        min_val = min_val_list[rand_comp-1][i]
        max_val = max_val_list[rand_comp-1][i]
        feature_val = random.uniform(min_val, max_val)
        data_point.append(feature_val)

    data_point.append(rand_comp)

    return data_point

In [8]:
min_val_list, max_val_list = get_feature_val_range(x, y)
#print(min_val_list, max_val_list)

data_final = []
for i in range(NUM_DATA_POINTS) : 
    rand_data_point = generate_one_data_point(min_val_list, max_val_list)
    data_final.append(rand_data_point)

df_final = pd.DataFrame(data_final, columns=FEATURE_LABELS)
#df_final.head()

df_final.to_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset_synthetic.csv', mode = 'w', index=False)

In [9]:
df_synth = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset_synthetic.csv', error_bad_lines=False).fillna(0)
df_synth.shape[0]

100000

In [10]:
x_synth = df_synth.drop('next_month_plan', axis='columns')
y_synth = df_synth['next_month_plan']
print(y_synth)

0        7
1        2
2        1
3        3
4        3
        ..
99995    4
99996    4
99997    6
99998    3
99999    3
Name: next_month_plan, Length: 100000, dtype: int64


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x_synth, y_synth, random_state = 0) 

In [12]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

model_xg = xgb.XGBClassifier()

model_xg.fit(x_train, y_train)
print('Prediction Score : ', model_xg.score(x_test, y_test))
xg_predictions = model_xg.predict(x_test)
cm_xg = confusion_matrix(y_test, xg_predictions)
print('Confusion matrix :\n', cm_xg)
print('Clasification Report :\n', classification_report(y_test, xg_predictions))

Prediction Score :  0.94048
Confusion matrix :
 [[2688    1  242    9   25   72    0   43]
 [  49 2897   74    4   47   43    0   25]
 [  68    1 2873    6   45  113    0   26]
 [   0    2    0 3039   51   31    0    0]
 [   0    0    0    0 2847  283    0    0]
 [   0    0    1    0    3 3104    0    0]
 [   0    0    0    0    0    0 3075    0]
 [   0    0    0   31  109   84    0 2989]]
Clasification Report :
               precision    recall  f1-score   support

           1       0.96      0.87      0.91      3080
           2       1.00      0.92      0.96      3139
           3       0.90      0.92      0.91      3132
           4       0.98      0.97      0.98      3123
           5       0.91      0.91      0.91      3130
           6       0.83      1.00      0.91      3108
           7       1.00      1.00      1.00      3075
           8       0.97      0.93      0.95      3213

    accuracy                           0.94     25000
   macro avg       0.94      0.94      0.

In [13]:
# print('Prediction Score : ', model_xg.score(x, y))
# xg_predictions = model_xg.predict(x)
# cm_xg = confusion_matrix(y, xg_predictions)
# print('Confusion matrix :\n', cm_xg)
# print('Clasification Report :\n', classification_report(y, xg_predictions))

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_mlp = MLPClassifier(max_iter=500, activation='relu', hidden_layer_sizes=(1000,8))
print('Model MLP:\n', model_mlp)

# model_mlp.fit(x_train, y_train)
# print('Prediction Score of MLP: ', model_mlp.score(x_test, y_test))
# mlp_predictions = model_mlp.predict(x_test)
# cm_mlp = confusion_matrix(y_test, mlp_predictions)
# print('Confusion matrix of MLP:\n', cm_mlp)
# print('Clasification Report of MLP:\n',classification_report(y_test, mlp_predictions))

Model MLP:
 MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1000, 8), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


In [15]:
# print('Prediction Score of MLP: ', model_mlp.score(x, y))
# mlp_predictions = model_mlp.predict(x)
# cm_mlp = confusion_matrix(y, mlp_predictions)
# print('Confusion matrix of MLP:\n', cm_mlp)
# print('Clasification Report of MLP:\n',classification_report(y, mlp_predictions))

In [16]:
df_t = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/test_dataset.csv', error_bad_lines=False).fillna(0)

df_test = df_t.drop('primary_identifier', axis='columns')

df_test["device_type"] = LabelEncoder().fit_transform(df_test["device_type"].astype(str))
df_test["device_category"] = LabelEncoder().fit_transform(df_test["device_category"].astype(str))
df_test["gender"] = LabelEncoder().fit_transform(df_test["gender"].astype(str))
df_test["district_name"] = LabelEncoder().fit_transform(df_test["district_name"].astype(str))
df_test["age_group"] = LabelEncoder().fit_transform(df_test["age_group"].astype(str))

#df_test = df_test.apply(LabelEncoder().fit_transform)

df_test.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,3,6,2,4,3,4,689.42,148.0,0.000000,148.146634,37.037382,4.0,38.251243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0
1,3,6,2,4,3,7,799.93,300.0,0.180222,119.466825,10.027957,30.0,24.125172,0.0,0.0,0.0,0.0,0.0,0.0,107.0,16.266667,0.0,21.0,5.107143,4.924111,0.0,0.0,0.0,0.0
2,1,1,2,4,1,20,1538.45,8215.0,0.000000,4048.511104,2053.950266,4.0,932.358667,0.0,0.0,0.0,0.0,0.0,0.0,108.0,90.000000,0.0,8.0,13.558333,16.397071,0.0,0.0,110.0,2.0
3,3,6,1,16,3,10,995.75,11613.0,0.000000,1057.448432,430.118721,27.0,268.457304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416667,0.0,2.0,0.333333,0.087355,0.0,0.0,0.0,0.0
4,3,6,2,10,2,154,553.61,26505.0,28.515345,2490.832742,883.505624,30.0,660.353290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,350.0,1.0


In [17]:
# xg_predictions_test = model_xg.predict(df_test)

# df_test['next_month_plan'] = xg_predictions_test
# df_test['primary_identifier'] = df_t['primary_identifier']

# df_test = df_test[['primary_identifier','next_month_plan']]
# df_test.head()

In [18]:
# df_test.to_csv('/content/drive/MyDrive/Projects/ADLHackothon/submission.csv', mode = 'w', index=False)